In [ ]:
from snowflake import connector
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv('/Users/peter/.env')
username = os.getenv("SNOWFLAKE_USERNAME")
password=os.getenv("SNOWFLAKE_PASSWORD")
account=os.getenv("SNOWFLAKE_ACCOUNT")
role=os.getenv("SNOWFLAKE_ROLE"), 
warehouse=os.getenv("SNOWFLAKE_WAREHOUSE")
print(username, password, account, role, warehouse)

: 

In [4]:
# establish Snowflake connection
connection = connector.connect(user=os.getenv("SNOWFLAKE_USERNAME"), 
                  password=os.getenv("SNOWFLAKE_PASSWORD"), 
                  account=os.getenv("SNOWFLAKE_ACCOUNT"), 
                  role=os.getenv("SNOWFLAKE_ROLE"), 
                  warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
                  )

In [7]:
pd.read_sql('''
            with bodod as (
            select * 
            from disco_core.disco_mixpanel.bod_individual_conversions_overdelivery as b 
                inner join disco_mixpanel.master_conversions as mc on mc.unique_key = b.conversion_unique_key
            where bod_spend > 0 
            ),

            prelim as (
            select  hdc.property_brand_tier:value::string as tier,
                    ldp.merchant as publisher,
                    --ldp.date,
                    --date_trunc('day', bodod.conversion_source_time) as event_date,
                    cb.name as advertiser,
                    --bodod.brand_id,
                    --cb.core_brand_id,
                    --bodod.driving_brand,
                    sum(widget_loads_d) as total_discofeed_loads_pub_ad,
                    sum(bod_spend) as total_ad_spend_pub_ad,
                    count(distinct conversion_unique_key) as total_conversions_pub_ad
                    --case when bodod.driving_brand is null then 0 
                    --  when ldp.merchant = bodod.driving_brand then 0
                    --  else 1
                    --  end as pub_check
                    
            from disco_core.disco_mixpanel.loads_displays_performance as ldp
                left join bodod on bodod.driving_brand = ldp.merchant and date_trunc('day', bodod.conversion_source_time) = ldp.date
                left join postgres.core_brand as cb on bodod.brand_id = cb.core_brand_id
                left join postgres.core_brand as cb2 on cb2.name = ldp.merchant 
                left join postgres.core_publisher as cp on cp.brand_id = cb2.core_brand_id
                left join hubspot_data.companies as hdc on hdc.property_disco_id:value::string = cp.remote_id
            where ldp.date >= '2024-01-01' --date_trunc('day', current_timestamp) - interval '7 days'
            group by 1, 2,3 --tier, publisher, advertiser
            order by publisher, total_ad_spend_pub_ad
            --order by pub_check desc
            ),
            final as (
            select tier, 
                    publisher, 
                    ARRAY_AGG(advertiser) WITHIN GROUP (ORDER BY total_ad_spend_pub_ad desc) as advertiser_list, 
                    sum(total_discofeed_loads_pub_ad) as total_discofeed_loads,
                    sum(total_ad_spend_pub_ad) as total_ad_spend_billable,
                    sum(total_conversions_pub_ad) as total_conversions_count,
                    case when sum(total_discofeed_loads_pub_ad) = 0 then 0 || '%'
                        else round((sum(total_conversions_pub_ad) / sum(total_discofeed_loads_pub_ad)) *100, 2) || '%'
                        end as cvr,
                    case when sum(total_discofeed_loads_pub_ad) = 0 then 0
                        else sum(total_ad_spend_pub_ad) / sum(total_discofeed_loads_pub_ad) 
                        end as rpl,
                    ARRAY_AGG(total_ad_spend_pub_ad) WITHIN GROUP (ORDER BY total_ad_spend_pub_ad desc) as ad_spend_list 
            from prelim
            where tier is not null
            group by 1, 2 
            order by 5 desc
            )
            select * 
            from final
            where total_ad_spend_billable is not null and total_discofeed_loads > 100
            order by 4 desc;
            '''
            , connection)

/var/folders/nn/ckstnx1110j6y5fknhgmvlvr0000gn/T/ipykernel_16311/2471446239.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('''


DatabaseError: Execution failed on sql '
            with bodod as (
            select * 
            from disco_core.disco_mixpanel.bod_individual_conversions_overdelivery as b 
                inner join disco_mixpanel.master_conversions as mc on mc.unique_key = b.conversion_unique_key
            where bod_spend > 0 
            ),

            prelim as (
            select  hdc.property_brand_tier:value::string as tier,
                    ldp.merchant as publisher,
                    --ldp.date,
                    --date_trunc('day', bodod.conversion_source_time) as event_date,
                    cb.name as advertiser,
                    --bodod.brand_id,
                    --cb.core_brand_id,
                    --bodod.driving_brand,
                    sum(widget_loads_d) as total_discofeed_loads_pub_ad,
                    sum(bod_spend) as total_ad_spend_pub_ad,
                    count(distinct conversion_unique_key) as total_conversions_pub_ad
                    --

/var/folders/nn/ckstnx1110j6y5fknhgmvlvr0000gn/T/ipykernel_16311/1519572421.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('''


,BRAND_ID,MAX_BUDGET,TARGET_CPA,TARGET_CPO,CONVERSION_SOURCE_TIME,CONVERSION_ORDER_TIME,CONVERSION_UNIQUE_KEY,MON_YEAR,BOD_SPEND_TILL_DATE,MAX_BUDGET_MONTH,...,NUM_BRANDS_EXCLUDED,EXPERIMENT_EXCLUSIONS,BRAND_VIEW_ORDER,DISTRIBUTION_PUBLISHER_ID,DISTRIBUTION_PARTNER_REMOTE_ID,UNIQUE_KEY,ROW_NUM,PUBLISHER,ID,IDENTIFIER
0,4278,3000.0,NaN,15.0,2023-10-25 19:01:42,2023-11-03 20:05:29,5049285869686,112023,330.0,3000.0,...,0.0,0.0,0.0,None,None,5049285869686,1,None,None,None
1,2215,5000.0,NaN,25.0,2023-11-17 03:17:17,2023-12-01 02:39:51,5681915265269,122023,25.0,5000.0,...,0.0,0.0,0.0,None,None,5681915265269,1,None,None,None
2,3990,25000.0,45.0,NaN,2023-10-30 20:35:57,2023-10-30 21:02:42,5720060657736,102023,7359.0,25000.0,...,0.0,0.0,1.0,None,None,5720060657736,1,None,None,None
3,1613,5000.0,20.0,NaN,2023-10-25 22:40:04,2023-10-29 04:04:39,5007145828488,102023,207.0,5000.0,...,0.0,0.0,NaN,None,None,5007145828488,1,None,None,None
4,2385,5000.0,NaN,15.0,2022-10-03 13:25:48,2022-10-12 19:32:52,4652733136998,102022,1560.0,5000.0,...,0.0,0.0,0.0,None,None,4652733136998,1,4adc7081-c05e-4522-9126-f7b871962af2,None,None
5,2345,20000.0,18.0,NaN,2023-11-01 23:20:53,2023-11-16 05:15:57,5213257039938,112023,734.0,20000.0,...,0.0,0.0,0.0,None,None,5213257039938,1,None,None,None
6,4380,5000.0,10.0,NaN,2024-01-12 15:44:23,2024-01-12 17:27:03,5391209136322,12024,1735.0,5000.0,...,0.0,0.0,0.0,None,None,5391209136322,1,None,None,None
7,4310,3000.0,NaN,8.0,2023-11-14 11:20:17,2023-11-24 12:04:31,5482272129251,112023,638.0,3000.0,...,0.0,0.0,0.0,None,None,5482272129251,1,None,None,None
8,2748,75000.0,NaN,10.0,2023-05-17 00:54:51,2023-05-31 14:53:14,5520533192766,52023,1765.0,75000.0,...,0.0,0.0,1.0,None,None,5520533192766,1,eaa591bf-63d6-4400-86ff-40e928af8c03,None,None
9,4192,25000.0,NaN,15.0,2023-10-12 05:03:51,2023-10-20 17:17:37,5138679759001,102023,2430.0,25000.0,...,0.0,0.0,1.0,None,None,5138679759001,1,None,None,None


In [2]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv('/Users/peter/.env')

True

In [3]:
credentials = {'user': os.getenv("SNOWFLAKE_USERNAME"), 
               'password': os.getenv("SNOWFLAKE_PASSWORD"), 
               'account': os.getenv("SNOWFLAKE_ACCOUNT"), 
               'role':os.getenv("SNOWFLAKE_ROLE"), 
               'warehouse': os.getenv("SNOWFLAKE_WAREHOUSE"),
}

In [5]:
engine = create_engine(URL(
    account = credentials['account'],
    user = credentials['user'],
    password = credentials['password'],
    warehouse = credentials['warehouse'],
    role = credentials['role']
))

con = engine.connect()

In [7]:
pd.read_sql('''
select * 
from disco_core.disco_mixpanel.bod_individual_conversions_overdelivery as b 
    inner join disco_core.disco_mixpanel.master_conversions as mc on mc.unique_key = b.conversion_unique_key
where bod_spend > 0
limit 25;
            '''
            , con)

/var/folders/nn/ckstnx1110j6y5fknhgmvlvr0000gn/T/ipykernel_19327/1332400254.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('''


AttributeError: 'Connection' object has no attribute 'cursor'